In [ ]:
import pandas as pd
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

# JOINS

[1378. Replace Employee ID With The Unique Identifier](https://leetcode.com/problems/replace-employee-id-with-the-unique-identifier/description/?envType=study-plan-v2&envId=top-sql-50)

In [ ]:
data = [[1, 'Alice'], [7, 'Bob'], [11, 'Meir'], [90, 'Winston'], [3, 'Jonathan']]
employees = pd.DataFrame(data, columns=['id', 'name']).astype({'id':'int64', 'name':'object'})
data = [[3, 1], [11, 2], [90, 3]]
employee_uni = pd.DataFrame(data, columns=['id', 'unique_id']).astype({'id':'int64', 'unique_id':'int64'})

In [ ]:
employees

,id,name
0,1,Alice
1,7,Bob
2,11,Meir
3,90,Winston
4,3,Jonathan


In [ ]:
employee_uni

,id,unique_id
0,3,1
1,11,2
2,90,3


In [ ]:
df = pd.merge(employees, employee_uni, how='left')

In [ ]:
df[['unique_id','name']].fillna("null")

,unique_id,name
0,null,Alice
1,null,Bob
2,2.0,Meir
3,3.0,Winston
4,1.0,Jonathan


In [ ]:
spark_employee = spark.createDataFrame(employees)
spark_employee.show()

+---+--------+
| id|    name|
+---+--------+
|  1|   Alice|
|  7|     Bob|
| 11|    Meir|
| 90| Winston|
|  3|Jonathan|
+---+--------+



In [ ]:
spark_employee_uni_df = spark.createDataFrame(employee_uni)
spark_employee_uni_df.show(truncate=False)

+---+---------+
|id |unique_id|
+---+---------+
|3  |1        |
|11 |2        |
|90 |3        |
+---+---------+



In [ ]:
spark_employee.join(spark_employee_uni_df, on=['id'], how='left').select('unique_id','name').show()

+---------+--------+
|unique_id|    name|
+---------+--------+
|     null|   Alice|
|     null|     Bob|
|        2|    Meir|
|        3| Winston|
|        1|Jonathan|
+---------+--------+



In [ ]:
spark_employee.createOrReplaceTempView('employees')
spark_employee_uni_df.createOrReplaceTempView('employee_uni')

spark.sql('''
select unique_id, name
from employees
left join employee_uni 
on employees.id = employee_uni.id;
''').show()

+---------+--------+
|unique_id|    name|
+---------+--------+
|     null|   Alice|
|     null|     Bob|
|        2|    Meir|
|        3| Winston|
|        1|Jonathan|
+---------+--------+



[1068. Product Sales Analysis I](https://leetcode.com/problems/product-sales-analysis-i/description/?envType=study-plan-v2&envId=top-sql-50)

In [ ]:
data = [[1, 100, 2008, 10, 5000], [2, 100, 2009, 12, 5000], [7, 200, 2011, 15, 9000]]
sales = pd.DataFrame(data, columns=['sale_id', 'product_id', 'year', 'quantity', 'price']).astype({'sale_id':'Int64', 'product_id':'Int64', 'year':'Int64', 'quantity':'Int64', 'price':'Int64'})
data = [[100, 'Nokia'], [200, 'Apple'], [300, 'Samsung']]
product = pd.DataFrame(data, columns=['product_id', 'product_name']).astype({'product_id':'Int64', 'product_name':'object'})

In [ ]:
output = pd.merge(sales, product, left_on='product_id', right_on='product_id', how= 'left')


In [ ]:
output[['product_name','year','price']]

,product_name,year,price
0,Nokia,2008,5000
1,Nokia,2009,5000
2,Apple,2011,9000


In [ ]:
spark_sales = spark.createDataFrame(sales)
spark_sales.show()

+-------+----------+----+--------+-----+
|sale_id|product_id|year|quantity|price|
+-------+----------+----+--------+-----+
|      1|       100|2008|      10| 5000|
|      2|       100|2009|      12| 5000|
|      7|       200|2011|      15| 9000|
+-------+----------+----+--------+-----+



In [ ]:
spark_product_df = spark.createDataFrame(product)
spark_product_df.show(truncate=False)

+----------+------------+
|product_id|product_name|
+----------+------------+
|100       |Nokia       |
|200       |Apple       |
|300       |Samsung     |
+----------+------------+



In [ ]:
spark_sales\
    .join(spark_product_df,on=['product_id'],how='left')\
    .select('product_name','year','price')\
    .show()

+------------+----+-----+
|product_name|year|price|
+------------+----+-----+
|       Nokia|2008| 5000|
|       Nokia|2009| 5000|
|       Apple|2011| 9000|
+------------+----+-----+



In [ ]:
spark_sales.createOrReplaceTempView('sales')
spark_product_df.createOrReplaceTempView('product')

spark.sql('''
select product_name, year, price
from sales
left join product
on sales.product_id=product.product_id;
''').show()

+------------+----+-----+
|product_name|year|price|
+------------+----+-----+
|       Nokia|2008| 5000|
|       Nokia|2009| 5000|
|       Apple|2011| 9000|
+------------+----+-----+



[1581. Customer Who Visited but Did Not Make Any Transactions](https://leetcode.com/problems/customer-who-visited-but-did-not-make-any-transactions/description/?envType=study-plan-v2&envId=top-sql-50)

In [ ]:
import pandas as pd

data = [[1, 23], [2, 9], [4, 30], [5, 54], [6, 96], [7, 54], [8, 54]]
visits = pd.DataFrame(data, columns=['visit_id', 'customer_id']).astype({'visit_id':'Int64', 'customer_id':'Int64'})
data = [[2, 5, 310], [3, 5, 300], [9, 5, 200], [12, 1, 910], [13, 2, 970]]
transactions = pd.DataFrame(data, columns=['transaction_id', 'visit_id', 'amount']).astype({'transaction_id':'Int64', 'visit_id':'Int64', 'amount':'Int64'})

In [ ]:
merged = pd.merge(visits, transactions, on='visit_id', how='left')

In [ ]:
filtering_null_transaction = merged[merged['transaction_id'].isna()]

In [ ]:
output = filtering_null_transaction.groupby('customer_id').size().reset_index(name='count_no_transaction_visits')

In [ ]:
output

,customer_id,count_no_transaction_visits
0,30,1
1,54,2
2,96,1


In [ ]:
visits_df = spark.createDataFrame(visits)
visits_df.show(truncate=False)

+--------+-----------+
|visit_id|customer_id|
+--------+-----------+
|1       |23         |
|2       |9          |
|4       |30         |
|5       |54         |
|6       |96         |
|7       |54         |
|8       |54         |
+--------+-----------+



In [ ]:
transactions_df = spark.createDataFrame(transactions)
transactions_df.show(truncate=False)

+--------------+--------+------+
|transaction_id|visit_id|amount|
+--------------+--------+------+
|2             |5       |310   |
|3             |5       |300   |
|9             |5       |200   |
|12            |1       |910   |
|13            |2       |970   |
+--------------+--------+------+



In [ ]:
visits_df\
    .join(transactions_df,on=['visit_id'],how='left')\
    .filter(F.col('transaction_id').isNull())\
    .groupBy(F.col('customer_id'))\
    .agg(F.count('customer_id').alias('count_no_trans'))\
    .show()

+-----------+--------------+
|customer_id|count_no_trans|
+-----------+--------------+
|         30|             1|
|         96|             1|
|         54|             2|
+-----------+--------------+



In [ ]:
visits_df.createOrReplaceTempView('visits')
transactions_df.createOrReplaceTempView('transactions')

spark.sql('''
select customer_id, count(customer_id) as count_no_trans
from visits 
left join transactions
on transactions.visit_id=visits.visit_id
where transaction_id is null
group by customer_id;
''').show()

+-----------+--------------+
|customer_id|count_no_trans|
+-----------+--------------+
|         30|             1|
|         96|             1|
|         54|             2|
+-----------+--------------+



[197. Rising Temperature](https://leetcode.com/problems/rising-temperature/description/?envType=study-plan-v2&envId=top-sql-50)

In [ ]:


data = [[1, '2015-01-01', 10], [2, '2015-01-02', 25], [3, '2015-01-03', 20], [4, '2015-01-04', 30]]
weather = pd.DataFrame(data, columns=['id', 'recordDate', 'temperature']).astype({'id':'Int64', 'recordDate':'datetime64[ns]', 'temperature':'Int64'})

In [ ]:
merged = weather.merge(
    weather,
    how='inner',
    left_on=weather['recordDate'] - pd.Timedelta(days=1),
    right_on='recordDate',
    suffixes=('_w1', '_w2')
)


In [ ]:
merged

,recordDate,id_w1,recordDate_w1,temperature_w1,id_w2,recordDate_w2,temperature_w2
0,2015-01-01,2,2015-01-02,25,1,2015-01-01,10
1,2015-01-02,3,2015-01-03,20,2,2015-01-02,25
2,2015-01-03,4,2015-01-04,30,3,2015-01-03,20


In [ ]:
filtered = merged[merged['temperature_w1'] > merged['temperature_w2']]
result = filtered['id_w1'].to_frame()


In [ ]:
result

,id_w1
0,2
2,4


In [ ]:
weather_df = spark.createDataFrame(weather)
weather_df.show(truncate=False)

+---+-------------------+-----------+
|id |recordDate         |temperature|
+---+-------------------+-----------+
|1  |2015-01-01 00:00:00|10         |
|2  |2015-01-02 00:00:00|25         |
|3  |2015-01-03 00:00:00|20         |
|4  |2015-01-04 00:00:00|30         |
+---+-------------------+-----------+



In [ ]:
(weather_df.alias("w1")
 .join(weather_df.alias("w2"), F.datediff(F.col("w1.recordDate"), F.col("w2.recordDate")) == 1, "inner")
 .where("w1.temperature > w2.temperature")
 .select("w1.id").show())

+---+
| id|
+---+
|  2|
|  4|
+---+



In [ ]:
# in Spark SQL
weather_df.createOrReplaceTempView('weather')
spark.sql('''
select
    w1.id
from
    weather w1
    join weather w2 on datediff(w1.recordDate, w2.recordDate) = 1
where
    w1.temperature > w2.temperature
''').show()

+---+
| id|
+---+
|  2|
|  4|
+---+



[1661. Average Time of Process per Machine](https://leetcode.com/problems/average-time-of-process-per-machine/description/?envType=study-plan-v2&envId=top-sql-50)

In [ ]:
data = [[0, 0, 'start', 0.712], [0, 0, 'end', 1.52], [0, 1, 'start', 3.14], [0, 1, 'end', 4.12], [1, 0, 'start', 0.55],
        [1, 0, 'end', 1.55], [1, 1, 'start', 0.43], [1, 1, 'end', 1.42], [2, 0, 'start', 4.1], [2, 0, 'end', 4.512],
        [2, 1, 'start', 2.5], [2, 1, 'end', 5]]
activity = pd.DataFrame(data, columns=['machine_id', 'process_id', 'activity_type', 'timestamp']).astype(
    {'machine_id': 'Int64', 'process_id': 'Int64', 'activity_type': 'object', 'timestamp': 'Float64'})

In [ ]:
activity

,machine_id,process_id,activity_type,timestamp
0,0,0,start,0.712
1,0,0,end,1.52
2,0,1,start,3.14
3,0,1,end,4.12
4,1,0,start,0.55
5,1,0,end,1.55
6,1,1,start,0.43
7,1,1,end,1.42
8,2,0,start,4.1
9,2,0,end,4.512


In [ ]:
pivoted = activity.pivot_table(
    index=['machine_id', 'process_id'],
    columns='activity_type',
    values='timestamp'
).reset_index()

In [ ]:
pivoted

activity_type,machine_id,process_id,end,start
0,0,0,1.52,0.712
1,0,1,4.12,3.14
2,1,0,1.55,0.55
3,1,1,1.42,0.43
4,2,0,4.512,4.1
5,2,1,5.0,2.5


In [ ]:
pivoted['duration'] = pivoted['end'] - pivoted['start']

In [ ]:
result = pivoted.groupby('machine_id')['duration'].mean().reset_index()
result.rename(columns={'duration': 'processing_time'}, inplace=True)
print(result)

   machine_id  processing_time
0           0            0.894
1           1            0.995
2           2            1.456


In [ ]:

activity_df = spark.createDataFrame(activity)
activity_df.show()

+----------+----------+-------------+---------+
|machine_id|process_id|activity_type|timestamp|
+----------+----------+-------------+---------+
|         0|         0|        start|    0.712|
|         0|         0|          end|     1.52|
|         0|         1|        start|     3.14|
|         0|         1|          end|     4.12|
|         1|         0|        start|     0.55|
|         1|         0|          end|     1.55|
|         1|         1|        start|     0.43|
|         1|         1|          end|     1.42|
|         2|         0|        start|      4.1|
|         2|         0|          end|    4.512|
|         2|         1|        start|      2.5|
|         2|         1|          end|      5.0|
+----------+----------+-------------+---------+



In [ ]:
result_spark = activity_df.join('',how='inner')

In [ ]:
start_df = activity_df.filter(F.col("activity_type")=='start').withColumnRenamed("timestamp","start_time")
end_df = activity_df.filter(F.col("activity_type") == "end") \
                    .withColumnRenamed("timestamp", "end_time")
        

In [ ]:
joined_df = start_df.join(
    end_df,
    on=["machine_id", "process_id"],
    how="inner"
)
processing_df = joined_df.withColumn("processing_time", F.col("end_time") - F.col("start_time"))

In [ ]:
result_df = processing_df.groupBy("machine_id") \
                         .agg(F.avg("processing_time").alias("processing_time")) \
                         .orderBy("machine_id")

# 5. Show the result
result_df.show()

In [ ]:
activity_df.createOrReplaceTempView('activity')
spark.sql('''
SELECT
    s.machine_id,
    AVG(e.timestamp - s.timestamp) AS processing_time
FROM
    activity s
JOIN
    activity e
    ON s.machine_id = e.machine_id
   AND s.process_id = e.process_id
   AND s.activity_type = 'start'
   AND e.activity_type = 'end'
GROUP BY
    s.machine_id
ORDER BY
    s.machine_id;
''').show()

[577. Employee Bonus](https://leetcode.com/problems/employee-bonus/description/?envType=study-plan-v2&envId=top-sql-50)

In [ ]:
import pandas as pd

data = [[3, 'Brad', None, 4000], [1, 'John', 3, 1000], [2, 'Dan', 3, 2000], [4, 'Thomas', 3, 4000]]
employee = pd.DataFrame(data, columns=['empId', 'name', 'supervisor', 'salary']).astype(
    {'empId': 'Int64', 'name': 'object', 'supervisor': 'Int64', 'salary': 'Int64'})
data2 = [[2, 500], [4, 2000]]
bonus = pd.DataFrame(data2, columns=['empId', 'bonus']).astype({'empId': 'Int64', 'bonus': 'Int64'})

In [ ]:
employee.head()

In [ ]:
bonus.head()

In [ ]:
intermediateResult = employee.merge(bonus, on='empId', how='left')

In [ ]:
intermediateResult = intermediateResult[
    (intermediateResult['bonus'] <= 1000) | (intermediateResult['bonus'].isna())
]

In [ ]:
result = intermediateResult[['name','bonus']]
result.head()

In [ ]:
schema_employee = StructType([
    StructField("empId", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("supervisor", IntegerType(), True),
    StructField("salary", IntegerType(), True)
])

spark_employee_df = spark.createDataFrame(data, schema_employee)
spark_bonus_df = spark.createDataFrame(bonus)

In [ ]:
spark_employee_df.join(spark_bonus_df, on=['empId'], how='left') \
    .filter((F.col('bonus') < 1000) | (F.col('bonus').isNull())) \
    .select('name', 'bonus').show()

In [ ]:
employee_df.createOrReplaceTempView('employee')
bonus_df.createOrReplaceTempView('bonus')

spark.sql('''
select name,bonus from employee
left join bonus
on employee.empId=bonus.empId
where bonus is null or bonus<1000;
''').show()

[1280. Students and Examinations](https://leetcode.com/problems/students-and-examinations/description/?envType=study-plan-v2&envId=top-sql-50)

In [ ]:
data = [[1, 'Alice'], [2, 'Bob'], [13, 'John'], [6, 'Alex']]
students = pd.DataFrame(data, columns=['student_id', 'student_name']).astype(
    {'student_id': 'Int64', 'student_name': 'object'})
data = [['Math'], ['Physics'], ['Programming']]
subjects = pd.DataFrame(data, columns=['subject_name']).astype({'subject_name': 'object'})
data = [[1, 'Math'], [1, 'Physics'], [1, 'Programming'], [2, 'Programming'], [1, 'Physics'], [1, 'Math'], [13, 'Math'],
        [13, 'Programming'], [13, 'Physics'], [2, 'Math'], [1, 'Math']]
examinations = pd.DataFrame(data, columns=['student_id', 'subject_name']).astype(
    {'student_id': 'Int64', 'subject_name': 'object'})

In [ ]:
students.head()

In [ ]:
subjects.head()

In [ ]:
examinations.head()

In [ ]:
students_subjects = students.assign(key = 1).merge(subjects.assign(key = 1), on='key').drop('key',axis=1)
students_subjects

In [ ]:
exam_count = examinations.groupby(['student_id','subject_name']).size().reset_index(name='attended_exam')
exam_count

In [ ]:
final_df = students_subjects.merge(exam_count, on=['student_id', 'subject_name'], how='left')
final_df['attended_exam'] = final_df['attended_exam'].fillna(0).astype(int)
final_df

In [ ]:
final_df = final_df.sort_values(by=['student_name', 'subject_name']).reset_index(drop=True)

print(final_df)

In [ ]:
students_df = spark.createDataFrame(students)
subjects_df = spark.createDataFrame(subjects)
examinations_df = spark.createDataFrame(examinations)

In [ ]:
students_subject_df = students_df.crossJoin(subjects_df)

# Step 2: Count how many times each student took each subject exam
exam_df = examinations_df.groupBy('student_id', 'subject_name') \
                         .agg(F.count('*').alias('attended_exam'))

# Step 3: Left join the counts to the full student-subject combinations
final_df = students_subject_df.join(
    exam_df,
    on=['student_id', 'subject_name'],
    how='left'
).withColumn(
    'attended_exam',
    F.coalesce(F.col('attended_exam'), F.lit(0))
)

# Step 4: Show the result ordered by student_id and subject_name
final_df.orderBy('student_id', 'subject_name').show()

In [ ]:
students_df.createOrReplaceTempView('students')
subjects_df.createOrReplaceTempView('subjects')
examinations_df.createOrReplaceTempView('examinations')

spark,sql('''
SELECT 
    s.student_id, 
    s.student_name, 
    sub.subject_name,
    COALESCE(COUNT(e.subject_name), 0) AS attended_exams
FROM 
    students s
CROSS JOIN 
    subjects sub
LEFT JOIN 
    examinations e 
    ON s.student_id = e.student_id AND sub.subject_name = e.subject_name
GROUP BY 
    s.student_id, s.student_name, sub.subject_name
ORDER BY 
    s.student_id, sub.subject_name;''').show()

[570. Managers with at Least 5 Direct Reports](https://leetcode.com/problems/managers-with-at-least-5-direct-reports/description/?envType=study-plan-v2&envId=top-sql-50)

In [ ]:
data = [[101, 'John', 'A', None], [102, 'Dan', 'A', 101], [103, 'James', 'A', 101], [104, 'Amy', 'A', 101], [105, 'Anne', 'A', 101], [106, 'Ron', 'B', 101]]
employee = pd.DataFrame(data, columns=['id', 'name', 'department', 'managerId']).astype({'id':'Int64', 'name':'object', 'department':'object', 'managerId':'Int64'})

In [ ]:
employee

In [ ]:
manager_counts = employee['managerId'].value_counts()

eligible_manager_id = manager_counts[manager_counts >=5 ].index

result_df = employee[employee['id'].isin(eligible_manager_id)]

result_df[['name']]

In [ ]:
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("department", StringType(), True),
    StructField("managerId", IntegerType(), True)
])

In [ ]:
employee_df = spark.createDataFrame(data,schema=schema)
employee_df.show(truncate=False)

In [ ]:
employee_df.select('name').filter(F.col('id').isin(
        list(employee_df.select('managerId').groupBy('managerId').agg(F.count(F.col('managerId')).alias('count')).filter(
        F.col('count') >= 5).select('managerId').rdd.map(lambda x:x[0]).collect())
)).show()

In [ ]:
employee_df.createOrReplaceTempView('employee')
spark.sql('''
select name
from employee
where id in (select managerId from employee group by managerId having count(managerId)>=5)
''').show()

[1934. Confirmation Rate](https://leetcode.com/problems/confirmation-rate/description/?envType=study-plan-v2&envId=top-sql-50)

In [ ]:
data = [[3, '2020-03-21 10:16:13'], [7, '2020-01-04 13:57:59'], [2, '2020-07-29 23:09:44'], [6, '2020-12-09 10:39:37']]
signups = pd.DataFrame(data, columns=['user_id', 'time_stamp']).astype(
    {'user_id': 'Int64', 'time_stamp': 'datetime64[ns]'})
data = [[3, '2021-01-06 03:30:46', 'timeout'], [3, '2021-07-14 14:00:00', 'timeout'],
        [7, '2021-06-12 11:57:29', 'confirmed'], [7, '2021-06-13 12:58:28', 'confirmed'],
        [7, '2021-06-14 13:59:27', 'confirmed'], [2, '2021-01-22 00:00:00', 'confirmed'],
        [2, '2021-02-28 23:59:59', 'timeout']]
confirmations = pd.DataFrame(data, columns=['user_id', 'time_stamp', 'action']).astype(
    {'user_id': 'Int64', 'time_stamp': 'datetime64[ns]', 'action': 'object'})

In [ ]:
signups

,user_id,time_stamp
0,3,2020-03-21 10:16:13
1,7,2020-01-04 13:57:59
2,2,2020-07-29 23:09:44
3,6,2020-12-09 10:39:37


In [ ]:
confirmations

,user_id,time_stamp,action
0,3,2021-01-06 03:30:46,timeout
1,3,2021-07-14 14:00:00,timeout
2,7,2021-06-12 11:57:29,confirmed
3,7,2021-06-13 12:58:28,confirmed
4,7,2021-06-14 13:59:27,confirmed
5,2,2021-01-22 00:00:00,confirmed
6,2,2021-02-28 23:59:59,timeout


In [ ]:
import numpy as np

In [ ]:
merged_df = pd.merge(signups, confirmations, on='user_id',how='left')

merged_df['confirmed_flag'] = np.where(merged_df['action']=='confirmed',1,0)


In [ ]:
merged_df

,user_id,time_stamp_x,time_stamp_y,action,confirmed_flag
0,3,2020-03-21 10:16:13,2021-01-06 03:30:46,timeout,0
1,3,2020-03-21 10:16:13,2021-07-14 14:00:00,timeout,0
2,7,2020-01-04 13:57:59,2021-06-12 11:57:29,confirmed,1
3,7,2020-01-04 13:57:59,2021-06-13 12:58:28,confirmed,1
4,7,2020-01-04 13:57:59,2021-06-14 13:59:27,confirmed,1
5,2,2020-07-29 23:09:44,2021-01-22 00:00:00,confirmed,1
6,2,2020-07-29 23:09:44,2021-02-28 23:59:59,timeout,0
7,6,2020-12-09 10:39:37,NaT,NaN,0


In [ ]:
result_df = merged_df.groupby('user_id')['confirmed_flag'] \
                     .mean().round(2) \
                     .reset_index() \
                     .rename(columns={'confirmed_flag': 'confirmation_rate'})

print(result_df)

   user_id  confirmation_rate
0        2                0.5
1        3                0.0
2        6                0.0
3        7                1.0


In [ ]:
signups_df = spark.createDataFrame(signups)
confirmations_df = spark.createDataFrame(confirmations)

In [ ]:
signups_df.join(confirmations_df, on=['user_id'], how='left').select('user_id', 'action') \
    .groupBy('user_id') \
    .agg(F.round(F.avg(F.when(F.col('action') == 'confirmed', 1).otherwise(0)), 2).alias('confirmation_rate')) \
    .show()

+-------+-----------------+
|user_id|confirmation_rate|
+-------+-----------------+
|      3|              0.0|
|      7|              1.0|
|      2|              0.5|
|      6|              0.0|
+-------+-----------------+



In [ ]:
signups_df.createOrReplaceTempView('signups')
confirmations_df.createOrReplaceTempView('confirmations')

spark.sql('''
select signups.user_id, round(avg(case when action='confirmed' then 1 else 0 end),2) as confirmation_rate 
from signups
left join confirmations
on signups.user_id = confirmations.user_id
group by signups.user_id
''').show()

+-------+-----------------+
|user_id|confirmation_rate|
+-------+-----------------+
|      3|              0.0|
|      7|              1.0|
|      2|              0.5|
|      6|              0.0|
+-------+-----------------+

